In [49]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [50]:
from shutil import copy2

In [51]:
import matplotlib.pyplot as plt

In [52]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
labels = []
dist = []
for race in races:
    for gender in genders:
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [53]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.23402979969978333


In [54]:
races = ['African','Asian','Caucasian','Indian']
genders = ['Man','Woman']
accuracies = {}
for race in races:
    for gender in genders:
        accuracies[f"{race}_{gender}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_{gender}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TN'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FN'] += 1
            else:
                with open(f"./results/{race}_{gender}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}_{gender}"]['TP'] += 1
                else:
                    accuracies[f"{race}_{gender}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_{gender}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [55]:
print(accuracies)

{'African_Man': {'TP': 2240, 'TN': 699, 'FP': 770, 'FN': 2170}, 'African_Woman': {'TP': 47, 'TN': 14, 'FP': 8, 'FN': 52}, 'Asian_Man': {'TP': 1635, 'TN': 493, 'FP': 372, 'FN': 1759}, 'Asian_Woman': {'TP': 619, 'TN': 253, 'FP': 187, 'FN': 682}, 'Caucasian_Man': {'TP': 1716, 'TN': 537, 'FP': 202, 'FN': 2050}, 'Caucasian_Woman': {'TP': 552, 'TN': 195, 'FP': 67, 'FN': 681}, 'Indian_Man': {'TP': 1890, 'TN': 425, 'FP': 479, 'FN': 1838}, 'Indian_Woman': {'TP': 539, 'TN': 146, 'FP': 153, 'FN': 529}}


In [56]:
gar = {}
far = {}
scores = {}
for race in races:
    for gender in genders:
        tp = accuracies[f"{race}_{gender}"]['TP']
        tn = accuracies[f"{race}_{gender}"]['TN']
        fp = accuracies[f"{race}_{gender}"]['FP']
        fn = accuracies[f"{race}_{gender}"]['FN']
        gar[f"{race}_{gender}"] = tp/(tn+tp)
        far[f"{race}_{gender}"] = fp/(fn+fp)
        scores[f"{race}_{gender}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [57]:
gar

{'African_Man': 0.7621640013610071,
 'African_Woman': 0.7704918032786885,
 'Asian_Man': 0.7683270676691729,
 'Asian_Woman': 0.7098623853211009,
 'Caucasian_Man': 0.7616511318242344,
 'Caucasian_Woman': 0.7389558232931727,
 'Indian_Man': 0.816414686825054,
 'Indian_Woman': 0.7868613138686131}

In [58]:
far

{'African_Man': 0.2619047619047619,
 'African_Woman': 0.13333333333333333,
 'Asian_Man': 0.17456593148756452,
 'Asian_Woman': 0.21518987341772153,
 'Caucasian_Man': 0.08969804618117229,
 'Caucasian_Woman': 0.08957219251336898,
 'Indian_Man': 0.20673284419507984,
 'Indian_Woman': 0.22434017595307917}

In [59]:
scores

{'African_Man': 75.01275727164484,
 'African_Woman': 81.81818181818181,
 'Asian_Man': 79.6900680911012,
 'Asian_Woman': 74.72716829408385,
 'Caucasian_Man': 83.59600443951166,
 'Caucasian_Woman': 82.47491638795987,
 'Indian_Man': 80.48359240069085,
 'Indian_Woman': 78.12728602779809}